# Laboratorijas darbs: Krītošas bumbas animācija

## Darba mērķis
Izveidot interaktīvu bumbas animāciju, kas simulē bumbas krišanu un atsperošanos saskaņā ar fizikas likumiem. Darbs ļaus izpētīt, kā dažādi parametri ietekmē bumbas kustību.

## Darba uzdevumi
1. Izmantojot slīdņus, mainīt šādus bumbas parametrus:
   - Bumbas krāsu
   - Bumbas izmēru
   - Kustības ātrumu
   - Sākuma augstumu
   - Sākuma pozīciju pa X asi
   - Elastības koeficientu (atsperošanās intensitāti)
2. Pēc katra parametra maiņas nospiest pogu "Atiestatīt bumbu", lai redzētu izmaiņas
3. Vērot, kā mainās bumbas kustības trajektorija atkarībā no izvēlētajiem parametriem

## Uzdevumi pētīšanai
1. Kā gravitācija ietekmē bumbas kustību?
2. Kā elastības koeficients ietekmē atsperošanos?
3. Kā bumbas izmērs ietekmē tās kustību?

**Piezīme:** Koda izpildei var būt nepieciešams dažas sekundes atkarībā no jūsu datora veiktspējas un interneta savienojuma ātruma.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import display, HTML, clear_output
import ipywidgets as widgets

In [ ]:
# Inicializācija
ball = None
ani = None

# Grafiskā vides iestatīšana
fig, ax = plt.subplots(figsize=(8, 6))
ax.set_xlim(0, 10)
ax.set_ylim(0, 10)
ax.set_aspect('equal')
ax.grid(True, linestyle='--', alpha=0.7)
plt.close(fig)

# Bumbas izveide
ball = plt.Circle((5, 5), 0.5, color='red', zorder=10)
ax.add_patch(ball)

# Fizikas parametri
pos = np.array([5.0, 5.0], dtype=float)  # Pozīcija
vel = np.array([1.5, 0.0], dtype=float)  # Ātrums
gravity = 0.2                            # Gravitācija
elasticity = 0.8                         # Elastība (sākuma vērtība)

In [ ]:
# Kontrolelementu izveide
color_picker = widgets.ColorPicker(
    value='red',
    description='Bumbas krāsa:',
    style={'description_width': 'initial'}
)

size_slider = widgets.FloatSlider(
    value=0.5,
    min=0.2,
    max=1.5,
    step=0.1,
    description='Bumbas izmērs:',
    style={'description_width': 'initial'}
)

speed_slider = widgets.FloatSlider(
    value=1.0,
    min=0.1,
    max=3.0,
    step=0.1,
    description='Ātrums:',
    style={'description_width': 'initial'}
)

height_slider = widgets.FloatSlider(
    value=5.0,
    min=1.0,
    max=9.0,
    step=0.5,
    description='Sākuma augstums:',
    style={'description_width': 'initial'}
)

xpos_slider = widgets.FloatSlider(
    value=5.0,
    min=1.0,
    max=9.0,
    step=0.5,
    description='X pozīcija:',
    style={'description_width': 'initial'}
)

elasticity_slider = widgets.FloatSlider(
    value=0.8,
    min=0.1,
    max=1.0,
    step=0.05,
    description='Elastība:',
    style={'description_width': 'initial'}
)

# Kontrolelementu izkārtojums
controls_top = widgets.HBox([
    widgets.VBox([color_picker, size_slider]),
    widgets.VBox([speed_slider, height_slider]),
    widgets.VBox([xpos_slider, elasticity_slider])
])
display(controls_top)

In [ ]:
# Reset poga un animācijas funkcijas
reset_button = widgets.Button(
    description='Atiestatīt bumbu',
    button_style='success',
    tooltip='Atiestata animāciju ar jauniem parametriem',
    layout={'width': '200px'}
)

def reset_ball(b):
    global pos, vel, ani, elasticity
    
    # Atjaunina parametrus
    pos = np.array([xpos_slider.value, height_slider.value], dtype=float)
    vel = np.array([1.5 * speed_slider.value, 0.0], dtype=float)
    elasticity = elasticity_slider.value
    
    # Atjaunina bumbu
    ball.center = pos
    ball.radius = size_slider.value
    ball.set_color(color_picker.value)
    
    # Pārtrauc veco animāciju
    if ani is not None:
        ani.event_source.stop()
    
    # Sāk jaunu animāciju
    start_animation()

def update(frame):
    global pos, vel
    
    # Fizikas aprēķini
    vel[1] -= gravity * speed_slider.value
    pos += vel * speed_slider.value
    
    # Sienu sadursmes
    current_radius = size_slider.value
    
    if pos[0] <= current_radius or pos[0] >= 10 - current_radius:
        vel[0] *= -elasticity
        pos[0] = np.clip(pos[0], current_radius, 10 - current_radius)
    
    if pos[1] <= current_radius:
        vel[1] *= -elasticity
        pos[1] = current_radius
    elif pos[1] >= 10 - current_radius:
        vel[1] *= -elasticity
        pos[1] = 10 - current_radius
    
    # Atjaunina bumbas pozīciju
    ball.center = pos
    ball.radius = current_radius
    ball.set_color(color_picker.value)
    
    return ball,

def start_animation():
    global ani
    clear_output(wait=True)
    display(controls_top)
    display(reset_button)
    ani = FuncAnimation(
        fig, 
        update, 
        frames=200, 
        interval=20, 
        blit=True,
        cache_frame_data=False
    )
    display(HTML(ani.to_jshtml()))

reset_button.on_click(reset_ball)
start_animation()